## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 1st Dec. 8th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-5-2024,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
78,PAST SALE,2024-12-09,Condo/Co-op,18501 Collins Ave #1903,Sunny Isles Beach,FL,33160.0,5050000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11576078,N,Y,25.947720,-80.120228
39,PAST SALE,2024-12-06,Condo/Co-op,17875 Collins Ave #3105,Sunny Isles Beach,FL,33160.0,2900000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11655999,N,Y,25.941257,-80.120772
88,PAST SALE,2024-12-06,Condo/Co-op,450 Alton Rd #2006,Miami Beach,FL,33139.0,1600000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/450-Alto...,MARMLS,A11520105,N,Y,25.773660,-80.141126
51,PAST SALE,2024-12-06,Condo/Co-op,18051 Biscayne Blvd Unit 301-1,Aventura,FL,33160.0,252000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/18051-Bisca...,MARMLS,A11639608,N,Y,25.942851,-80.145774
77,PAST SALE,2024-12-06,Condo/Co-op,2555 Collins Ave #2411,Miami Beach,FL,33140.0,1150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2555-Col...,MARMLS,A11562514,N,Y,25.801678,-80.125793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,PAST SALE,2024-12-02,Condo/Co-op,229 NW 10th Ave #2,Miami,FL,33128.0,180000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/229-NW-10th-Av...,MARMLS,A11657102,N,Y,25.776104,-80.211338
54,PAST SALE,2024-12-02,Condo/Co-op,19380 Collins Ave #214,Sunny Isles Beach,FL,33160.0,282500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11647952,N,Y,25.954860,-80.123290
55,PAST SALE,2024-12-02,Condo/Co-op,800 West Ave #824,Miami Beach,FL,33139.0,260000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/800-West...,MARMLS,A11644231,N,Y,25.778011,-80.142466
56,PAST SALE,2024-12-02,Condo/Co-op,850 N Miami Ave Unit W-1803,Miami,FL,33136.0,365000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/850-N-Miami-Av...,Beaches MLS,RX-11009177,N,Y,25.782554,-80.194525


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [10]:
df_top_ten = df.head(10)

## Collect Agent Information

In [11]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [12]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [13]:
response_df = pd.DataFrame(response_list)

In [14]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [15]:
df_top_ten = merged_df

## Current Week's Values

In [16]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

94


In [17]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$846,364


In [18]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$587


In [19]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$79,558,199


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [20]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
83
Input Previous Week Condo Average Sales Price:
651882
Input Previous Week Condo Average PSF:
465
Input Previous Week Condo Sales total (ex: 198_000_000)
54_100_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [21]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [22]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [23]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [24]:
pd.set_option('display.max_columns',None)

In [25]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange


In [26]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [27]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange
1,PAST SALE,2024-12-03,Condo/Co-op,7480 Miami Lakes Dr Unit G204,Miami Lakes,FL,33014.0,350000.0,2.0,2.0,VILLAS OF MIAMI LAKES CON,1080.0,NaN,1967.0,NaN,324.0,384.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,MARMLS,A11670058,N,Y,25.911458,-80.322283,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,Eduardo Arriaga,Miami New Realty,None,None,Rogelio Lucas,"Stratwell, LLC",None,None,2,blue
2,PAST SALE,2024-12-05,Condo/Co-op,4141 Nautilus Dr Unit 4B,Miami Beach,FL,33140.0,295000.0,1.0,1.5,EXCELSIOR CONDOMINIUM,760.0,NaN,1970.0,NaN,388.0,498.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,MARMLS,A11689213,N,Y,25.813871,-80.135298,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,Melissa Rossi,"Real Estate Craft, LLC.",Ricardo Guerrero,"Real Estate Craft, LLC.",Melissa Rossi,"Real Estate Craft, LLC.",None,None,3,blue
3,PAST SALE,2024-12-04,Condo/Co-op,22921 SW 88th Pl #105,Cutler Bay,FL,33190.0,300000.0,2.0,2.0,THE COURTS AT BAYSHORE II,817.0,NaN,2007.0,NaN,367.0,283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,MARMLS,A11681653,N,Y,25.557292,-80.333667,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,Neysa Morales Gonzalez,Martin Realty Group Corp,None,None,Diana Rodriguez,"ACE Florida Realty, LLC",None,None,4,blue
4,PAST SALE,2024-12-04,Condo/Co-op,10080 NW 9th Street Cir #206,Miami,FL,33172.0,380000.0,3.0,2.0,INDIAN LAKE VILLAGE II CO,1275.0,NaN,1979.0,NaN,298.0,413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,MARMLS,A11689136,N,Y,25.776608,-80.360619,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,None,None,None,None,None,None,None,None,5,blue
5,PAST SALE,2024-12-03,Condo/Co-op,75 NE 12th Ave #75,Homestead,FL,33030.0,260000.0,2.0,1.5,SEAGRAPE VILLAGE CONDO,1084.0,NaN,1982.0,NaN,240.0,160.0,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/75-NE-12th...,MARMLS,A11665458,N,Y,25.470912,-80.460445,https://www.redfin.com/FL/Homestead/75-NE-12th...,Cynthia Fernandez Ocasio,Lifestyle International Realty,None,None,Clyfford Philippe,United Realty Group Inc,None,None,6,blue
6,PAST SALE,2024-12-06,Condo/Co-op,6904 N Kendall Dr Unit F203,Pinecrest,FL,33156.0,317900.0,2.0,1.0,DADELAND GROVE CONDO,1057.0,NaN,1970.0,NaN,301.0,541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,MARMLS,A11682955,N,Y,25.687432,-80.307818,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,Moises Saca,Best American Realty Corp,None,None,Marianela Brown,The Keyes Company,None,None,7,blue
7,PAST SALE,2024-12-03,Condo/Co-op,7661 NW 107th Ave #210,Doral,FL,33178.0,408000.0,2.0,2.0,GRAND BAY COMMONS SOUTH,1076.0,NaN,2017.0,NaN,379.0,1064.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/7661-NW-107th-...,MARMLS,A11671771,N,Y,25.843155,-80.370114,https://www.redfin.com/FL/Doral/7661-NW-107th-...,Jennifer De Angelis,First Service Realty ERA,None,None,Franco Rizzo,Kendall Gables Realty Corp.,None,None,8,blue
8,PAST SALE,2024-12-02,Condo/Co-op,6175 W 20th Ave #214,Hialeah,FL,33012.0,275000.0,2.0,2.0,VILLA LUISA CONDO,890.0,NaN,1982.0,NaN,309.0,290.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6175-W

In [28]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [29]:
m.save('index.html')

## Data snagger

In [30]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_12092024


# CREATE TEMPLATE 

In [33]:
muni_set = set(df_top_ten['CITY'])

In [34]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [35]:
df_top_ten.reset_index(inplace=True,drop=True)

In [36]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [37]:
top_sale

'BRICKELL FLATIRON CONDO at 1000 Brickell Plz Ph 5105 in Miami'

In [38]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [39]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [40]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
1,PAST SALE,2024-12-03,Condo/Co-op,7480 Miami Lakes Dr Unit G204,Miami Lakes,FL,33014.0,350000.0,2.0,2.0,VILLAS OF MIAMI LAKES CON,1080.0,NaN,1967.0,NaN,324.0,384.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,MARMLS,A11670058,N,Y,25.911458,-80.322283,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,Eduardo Arriaga,Miami New Realty,None,None,Rogelio Lucas,"Stratwell, LLC",None,None,2,blue,VILLAS OF MIAMI LAKES CON at 7480 Miami Lakes ...
2,PAST SALE,2024-12-05,Condo/Co-op,4141 Nautilus Dr Unit 4B,Miami Beach,FL,33140.0,295000.0,1.0,1.5,EXCELSIOR CONDOMINIUM,760.0,NaN,1970.0,NaN,388.0,498.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,MARMLS,A11689213,N,Y,25.813871,-80.135298,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,Melissa Rossi,"Real Estate Craft, LLC.",Ricardo Guerrero,"Real Estate Craft, LLC.",Melissa Rossi,"Real Estate Craft, LLC.",None,None,3,blue,EXCELSIOR CONDOMINIUM at 4141 Nautilus Dr Unit...
3,PAST SALE,2024-12-04,Condo/Co-op,22921 SW 88th Pl #105,Cutler Bay,FL,33190.0,300000.0,2.0,2.0,THE COURTS AT BAYSHORE II,817.0,NaN,2007.0,NaN,367.0,283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,MARMLS,A11681653,N,Y,25.557292,-80.333667,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,Neysa Morales Gonzalez,Martin Realty Group Corp,None,None,Diana Rodriguez,"ACE Florida Realty, LLC",None,None,4,blue,THE COURTS AT BAYSHORE II at 22921 SW 88th Pl ...
4,PAST SALE,2024-12-04,Condo/Co-op,10080 NW 9th Street Cir #206,Miami,FL,33172.0,380000.0,3.0,2.0,INDIAN LAKE VILLAGE II CO,1275.0,NaN,1979.0,NaN,298.0,413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,MARMLS,A11689136,N,Y,25.776608,-80.360619,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,None,None,None,None,None,None,None,None,5,blue,INDIAN LAKE VILLAGE II CO at 10080 NW 9th Stre...
5,PAST SALE,2024-12-03,Condo/Co-op,75 NE 12th Ave #75,Homestead,FL,33030.0,260000.0,2.0,1.5,SEAGRAPE VILLAGE CONDO,1084.0,NaN,1982.0,NaN,240.0,160.0,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/75-NE-12th...,MARMLS,A11665458,N,Y,25.470912,-80.460445,https://www.redfin.com/FL/Homestead/75-NE-12th...,Cynthia Fernandez Ocasio,Lifestyle International Realty,None,None,Clyfford Philippe,United Realty Group Inc,None,None,6,blue,SEAGRAPE VILLAGE CONDO at 75 NE 12th Ave #75 i...
6,PAST SALE,2024-12-06,Condo/Co-op,6904 N Kendall Dr Unit F203,Pinecrest,FL,33156.0,317900.0,2.0,1.0,DADELAND GROVE CONDO,1057.0,NaN,1970.0,NaN,301.0,541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,MARMLS,A11682955,N,Y,25.687432,-80.307818,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,Moises Saca,Best American Realty Corp,None,None,Marianela Brown,The Keyes Company,None,None,7,blue,DADELAND GROVE CONDO at 6904 N Kendall Dr Unit...
7,PAST SALE,2024-12-03,Condo/Co-op,7661 NW 107th Ave #210,Doral,FL,33178.0,408000.0,2.0,2.0,GRAND BAY COMMONS SOUTH,1076.0,NaN,2017.0,NaN,379.0,1064.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/7661-NW-107th-...,MARMLS,A11671771,N,Y,25.843155,-80.370114

In [55]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: BRICKELL FLATIRON CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $252,000M to $1,600,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Homestead, Miami, Miami Lakes, Miami Beach, Doral, Hialeah, Cutler Bay, Pinecrest

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 94 condo sales totaling $79,558,199 million from Dec. 1st to Dec. 8th. The previous week, brokers closed 83 condo sales totaling $54,100,000.

Last week’s units sold for an average of $846,364, higher than the $651,882 average price 

In [56]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [57]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [58]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
1,PAST SALE,2024-12-03,Condo/Co-op,7480 Miami Lakes Dr Unit G204,Miami Lakes,FL,33014.0,350000.0,2.0,2.0,VILLAS OF MIAMI LAKES CON,1080.0,NaN,1967.0,NaN,324.0,384.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,MARMLS,A11670058,N,Y,25.911458,-80.322283,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,Eduardo Arriaga,Miami New Realty,None,None,Rogelio Lucas,"Stratwell, LLC",None,None,2,blue,VILLAS OF MIAMI LAKES CON at 7480 Miami Lakes ...
2,PAST SALE,2024-12-05,Condo/Co-op,4141 Nautilus Dr Unit 4B,Miami Beach,FL,33140.0,295000.0,1.0,1.5,EXCELSIOR CONDOMINIUM,760.0,NaN,1970.0,NaN,388.0,498.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,MARMLS,A11689213,N,Y,25.813871,-80.135298,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,Melissa Rossi,"Real Estate Craft, LLC.",Ricardo Guerrero,"Real Estate Craft, LLC.",Melissa Rossi,"Real Estate Craft, LLC.",None,None,3,blue,EXCELSIOR CONDOMINIUM at 4141 Nautilus Dr Unit...
3,PAST SALE,2024-12-04,Condo/Co-op,22921 SW 88th Pl #105,Cutler Bay,FL,33190.0,300000.0,2.0,2.0,THE COURTS AT BAYSHORE II,817.0,NaN,2007.0,NaN,367.0,283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,MARMLS,A11681653,N,Y,25.557292,-80.333667,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,Neysa Morales Gonzalez,Martin Realty Group Corp,None,None,Diana Rodriguez,"ACE Florida Realty, LLC",None,None,4,blue,THE COURTS AT BAYSHORE II at 22921 SW 88th Pl ...
4,PAST SALE,2024-12-04,Condo/Co-op,10080 NW 9th Street Cir #206,Miami,FL,33172.0,380000.0,3.0,2.0,INDIAN LAKE VILLAGE II CO,1275.0,NaN,1979.0,NaN,298.0,413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,MARMLS,A11689136,N,Y,25.776608,-80.360619,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,None,None,None,None,None,None,None,None,5,blue,INDIAN LAKE VILLAGE II CO at 10080 NW 9th Stre...
5,PAST SALE,2024-12-03,Condo/Co-op,75 NE 12th Ave #75,Homestead,FL,33030.0,260000.0,2.0,1.5,SEAGRAPE VILLAGE CONDO,1084.0,NaN,1982.0,NaN,240.0,160.0,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/75-NE-12th...,MARMLS,A11665458,N,Y,25.470912,-80.460445,https://www.redfin.com/FL/Homestead/75-NE-12th...,Cynthia Fernandez Ocasio,Lifestyle International Realty,None,None,Clyfford Philippe,United Realty Group Inc,None,None,6,blue,SEAGRAPE VILLAGE CONDO at 75 NE 12th Ave #75 i...
6,PAST SALE,2024-12-06,Condo/Co-op,6904 N Kendall Dr Unit F203,Pinecrest,FL,33156.0,317900.0,2.0,1.0,DADELAND GROVE CONDO,1057.0,NaN,1970.0,NaN,301.0,541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,MARMLS,A11682955,N,Y,25.687432,-80.307818,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,Moises Saca,Best American Realty Corp,None,None,Marianela Brown,The Keyes Company,None,None,7,blue,DADELAND GROVE CONDO at 6904 N Kendall Dr Unit...
7,PAST SALE,2024-12-03,Condo/Co-op,7661 NW 107th Ave #210,Doral,FL,33178.0,408000.0,2.0,2.0,GRAND BAY COMMONS SOUTH,1076.0,NaN,2017.0,NaN,379.0,1064.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/7661-NW-107th-...,MARMLS,A11671771,N,Y,25.843155,-80.370114

In [69]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: BRICKELL FLATIRON CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $252,000M to $1,600,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Homestead, Miami, Miami Lakes, Miami Beach, Doral, Hialeah, Cutler Bay, Pinecrest

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 94 condo sales totaling $79,558,199 million from Dec. 1st to Dec. 8th. The previous week, brokers closed 83 condo sales totaling $54,100,000.

Last week’s units sold for an average of $846,364, higher than the $651,882 average price 

In [64]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami/1000-Brickell-Plz-33131/unit-5105/home/79135675


In [48]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Lakes/7480-Miami-Lakes-Dr-33014/unit-G204/home/43264200


In [49]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Doral/8160-Geneva-Ct-33166/unit-509/home/43360155


In [50]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hialeah/6175-W-20th-Ave-33012/unit-214/home/42829649


In [51]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/1000-Brickell-Plz-33131/unit-5105/home/79135675


In [52]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Homestead/75-NE-12th-Ave-33030/unit-75/home/43350720


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-05,Condo/Co-op,1000 Brickell Plz Ph 5105,Miami,FL,33131.0,1600000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1236.0,NaN,2019.0,NaN,1294.0,3764.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11673873,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Patricia Bastidas,Fortune Christie's International Real Estate,None,None,Peter Prudenzano,"Compass Florida, LLC",None,None,1,orange,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
1,PAST SALE,2024-12-03,Condo/Co-op,7480 Miami Lakes Dr Unit G204,Miami Lakes,FL,33014.0,350000.0,2.0,2.0,VILLAS OF MIAMI LAKES CON,1080.0,NaN,1967.0,NaN,324.0,384.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,MARMLS,A11670058,N,Y,25.911458,-80.322283,https://www.redfin.com/FL/Miami-Lakes/7480-Mia...,Eduardo Arriaga,Miami New Realty,None,None,Rogelio Lucas,"Stratwell, LLC",None,None,2,blue,VILLAS OF MIAMI LAKES CON at 7480 Miami Lakes ...
2,PAST SALE,2024-12-05,Condo/Co-op,4141 Nautilus Dr Unit 4B,Miami Beach,FL,33140.0,295000.0,1.0,1.5,EXCELSIOR CONDOMINIUM,760.0,NaN,1970.0,NaN,388.0,498.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,MARMLS,A11689213,N,Y,25.813871,-80.135298,https://www.redfin.com/FL/Miami-Beach/4141-Nau...,Melissa Rossi,"Real Estate Craft, LLC.",Ricardo Guerrero,"Real Estate Craft, LLC.",Melissa Rossi,"Real Estate Craft, LLC.",None,None,3,blue,EXCELSIOR CONDOMINIUM at 4141 Nautilus Dr Unit...
3,PAST SALE,2024-12-04,Condo/Co-op,22921 SW 88th Pl #105,Cutler Bay,FL,33190.0,300000.0,2.0,2.0,THE COURTS AT BAYSHORE II,817.0,NaN,2007.0,NaN,367.0,283.0,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,MARMLS,A11681653,N,Y,25.557292,-80.333667,https://www.redfin.com/FL/Cutler-Bay/22921-SW-...,Neysa Morales Gonzalez,Martin Realty Group Corp,None,None,Diana Rodriguez,"ACE Florida Realty, LLC",None,None,4,blue,THE COURTS AT BAYSHORE II at 22921 SW 88th Pl ...
4,PAST SALE,2024-12-04,Condo/Co-op,10080 NW 9th Street Cir #206,Miami,FL,33172.0,380000.0,3.0,2.0,INDIAN LAKE VILLAGE II CO,1275.0,NaN,1979.0,NaN,298.0,413.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,MARMLS,A11689136,N,Y,25.776608,-80.360619,https://www.redfin.com/FL/Miami/10080-NW-9th-S...,None,None,None,None,None,None,None,None,5,blue,INDIAN LAKE VILLAGE II CO at 10080 NW 9th Stre...
5,PAST SALE,2024-12-03,Condo/Co-op,75 NE 12th Ave #75,Homestead,FL,33030.0,260000.0,2.0,1.5,SEAGRAPE VILLAGE CONDO,1084.0,NaN,1982.0,NaN,240.0,160.0,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/75-NE-12th...,MARMLS,A11665458,N,Y,25.470912,-80.460445,https://www.redfin.com/FL/Homestead/75-NE-12th...,Cynthia Fernandez Ocasio,Lifestyle International Realty,None,None,Clyfford Philippe,United Realty Group Inc,None,None,6,blue,SEAGRAPE VILLAGE CONDO at 75 NE 12th Ave #75 i...
6,PAST SALE,2024-12-06,Condo/Co-op,6904 N Kendall Dr Unit F203,Pinecrest,FL,33156.0,317900.0,2.0,1.0,DADELAND GROVE CONDO,1057.0,NaN,1970.0,NaN,301.0,541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,MARMLS,A11682955,N,Y,25.687432,-80.307818,https://www.redfin.com/FL/Pinecrest/6904-N-Ken...,Moises Saca,Best American Realty Corp,None,None,Marianela Brown,The Keyes Company,None,None,7,blue,DADELAND GROVE CONDO at 6904 N Kendall Dr Unit...
7,PAST SALE,2024-12-03,Condo/Co-op,7661 NW 107th Ave #210,Doral,FL,33178.0,408000.0,2.0,2.0,GRAND BAY COMMONS SOUTH,1076.0,NaN,2017.0,NaN,379.0,1064.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/7661-NW-107th-...,MARMLS,A11671771,N,Y,25.843155,-80.370114

## Time on Market Calculator

In [81]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 2) ## List (Earlier) date
date2 = datetime(2024, 12, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

62


## Clean CSV for Datawrapper Chart

In [70]:
chart_df = df_top_ten

In [71]:
chart_df = chart_df.fillna(" ")

In [72]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [73]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [74]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [75]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [79]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-05,1000 Brickell Plz Ph 5105 in Miami,"$1,600,000",2,2.5,"1,236",2019,"$1,294",MARMLS,A11673873,Patricia Bastidas Fortune Christie's Internati...,"Peter Prudenzano Compass Florida, LLC"
1,2024-12-03,7480 Miami Lakes Dr Unit G204 in Miami Lakes,"$350,000",2,2.0,"1,080",1967,$324,MARMLS,A11670058,Eduardo Arriaga Miami New Realty,"Rogelio Lucas Stratwell, LLC"
2,2024-12-05,4141 Nautilus Dr Unit 4B in Miami Beach,"$295,000",1,1.5,760,1970,$388,MARMLS,A11689213,"Melissa Rossi Real Estate Craft, LLC. Ricardo ...","Melissa Rossi Real Estate Craft, LLC."
3,2024-12-04,22921 SW 88th Pl #105 in Cutler Bay,"$300,000",2,2.0,817,2007,$367,MARMLS,A11681653,Neysa Morales Gonzalez Martin Realty Group Cor...,"Diana Rodriguez ACE Florida Realty, LLC"
4,2024-12-04,10080 NW 9th Street Cir #206 in Miami,"$380,000",3,2.0,"1,275",1979,$298,MARMLS,A11689136,,
5,2024-12-03,75 NE 12th Ave #75 in Homestead,"$260,000",2,1.5,"1,084",1982,$240,MARMLS,A11665458,Cynthia Fernandez Ocasio Lifestyle Internation...,Clyfford Philippe United Realty Group Inc
6,2024-12-06,6904 N Kendall Dr Unit F203 in Pinecrest,"$317,900",2,1.0,"1,057",1970,$301,MARMLS,A11682955,Moises Saca Best American Realty Corp,Marianela Brown The Keyes Company
7,2024-12-03,7661 NW 107th Ave #210 in Doral,"$408,000",2,2.0,"1,076",2017,$379,MARMLS,A11671771,Jennifer De Angelis First Service Realty ERA,Franco Rizzo Kendall Gables Realty Corp.
8,2024-12-02,6175 W 20th Ave #214 in Hialeah,"$275,000",2,2.0,890,1982,$309,MARMLS,A11669724,Frank Moya Miami New Realty,Jeffrey Roa Keller Williams Realty Internation...
9,2024-12-04,8160 Geneva Ct #509 in Doral,"$252,000",1,1.0,710,1984,$355,MARMLS,A11674882,Maria Galarraga Lifestyle International Realty...,Maria Teresa Neira The Agency Real Estate Firm...
